In [4]:
import os
from datetime import datetime, timedelta
import cdsapi
import xarray as xr
import pandas as pd
import numpy as np

# ------------------------------
# Configurações
# ------------------------------
output_dir = r'd:\Projetos\previsao-do-tempo\previsao'
os.makedirs(output_dir, exist_ok=True)

today = datetime.utcnow() - timedelta(hours=3)  # horário de Brasília
run_date = today.strftime('%Y%m%d_00Z')

# Arquivos de saída
target_file_hres = os.path.join(output_dir, f'ecmwf_rain_tp_run_{run_date}.grib2')
target_file_ext = os.path.join(output_dir, f'ecmwf_rain_tp_run_{run_date}_46d.grib2')

# ------------------------------
# Inicializa CDS API
# ------------------------------
c = cdsapi.Client()

# ------------------------------
# 1. Baixa ECMWF HRES (já funciona)
# ------------------------------
if not os.path.exists(target_file_hres):
    print(f"⬇️ Baixando ECMWF HRES: {target_file_hres}")
    try:
        c.retrieve(
            'seasonal-subseasonal',  # Dataset correto para HRES/Sub-Seasonal
            {
                'forecast_type': 'deterministic',
                'variable': 'total_precipitation',
                'time': '00:00',
                'start_date': today.strftime('%Y-%m-%d'),
                'end_date': today.strftime('%Y-%m-%d'),
                'leadtime_hour': list(range(0, 241, 3)),  # até 10 dias, 3h step
                'format': 'grib'
            },
            target_file_hres
        )
    except Exception as e:
        print(f"❌ Erro no download HRES: {e}")
        exit(1)
else:
    print("✅ ECMWF HRES já existe.")

# ------------------------------
# 2. Baixa ECMWF ENS Extended (46 dias)
# ------------------------------
if not os.path.exists(target_file_ext):
    print(f"⬇️ Baixando ECMWF ENS Extended (46 dias): {target_file_ext}")
    try:
        c.retrieve(
            'seasonal-subseasonal',  # Dataset correto para ENS Extended
            {
                'forecast_type': 'subseasonal',
                'variable': 'total_precipitation',
                'time': '00:00',
                'start_date': today.strftime('%Y-%m-%d'),
                'end_date': today.strftime('%Y-%m-%d'),
                'leadtime_hour': list(range(24, 1105, 24)),  # 1 a 46 dias
                'format': 'grib',
                'ensemble_member': 'all'
            },
            target_file_ext
        )
    except Exception as e:
        print(f"❌ Erro no download ENS Extended: {e}")
        exit(1)
else:
    print("✅ ENS Extended já existe.")

# ------------------------------
# 3. Processa GRIBs para acumulados
# ------------------------------
def process_grib(file_path):
    """Carrega GRIB e retorna DataFrame de precipitação diária"""
    ds = xr.open_dataset(file_path, engine='cfgrib')
    
    # Seleciona apenas total_precipitation
    if 'tp' in ds:
        tp = ds['tp']
    elif 'total_precipitation' in ds:
        tp = ds['total_precipitation']
    else:
        raise ValueError("Não encontrou variável de precipitação no GRIB.")

    # Converte para mm/dia
    tp_mm = tp * 1000.0

    # Agrupa por dia e soma para acumulado diário
    tp_daily = tp_mm.resample(time='1D').sum()
    
    return tp_daily

# HRES
tp_hres = process_grib(target_file_hres)

# ENS Extended
tp_ext = process_grib(target_file_ext)

# ------------------------------
# 4. Calcula acumulados
# ------------------------------
def acumulos(tp_series):
    """Calcula acumulados semanais, 30 e 46 dias"""
    weekly = tp_series.rolling(time=7, min_periods=1).sum()
    days30 = tp_series.rolling(time=30, min_periods=1).sum()
    days46 = tp_series.rolling(time=46, min_periods=1).sum()
    return weekly, days30, days46

weekly_hres, days30_hres, days46_hres = acumulos(tp_hres)
weekly_ext, days30_ext, days46_ext = acumulos(tp_ext)

# ------------------------------
# 5. Exemplo: salvar acumulados em CSV
# ------------------------------
weekly_ext.to_dataframe(name='weekly_tp').to_csv(os.path.join(output_dir, 'tp_weekly_ext.csv'))
days30_ext.to_dataframe(name='tp_30d').to_csv(os.path.join(output_dir, 'tp_30d_ext.csv'))
days46_ext.to_dataframe(name='tp_46d').to_csv(os.path.join(output_dir, 'tp_46d_ext.csv'))

print("✅ Processamento concluído. Arquivos de acumulado gerados.")

C:\Users\Heitor\AppData\Local\Temp\ipykernel_4988\3804934864.py:14: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow() - timedelta(hours=3)  # horário de Brasília


⬇️ Baixando ECMWF HRES: d:\Projetos\previsao-do-tempo\previsao\ecmwf_rain_tp_run_20251029_00Z.grib2
❌ Erro no download HRES: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/retrieve/v1/processes/seasonal-subseasonal
process not found
dataset seasonal-subseasonal not found
⬇️ Baixando ECMWF ENS Extended (46 dias): d:\Projetos\previsao-do-tempo\previsao\ecmwf_rain_tp_run_20251029_00Z_46d.grib2


Can't create file 'd:\\Projetos\\previsao-do-tempo\\previsao\\ecmwf_rain_tp_run_20251029_00Z.grib2.5b7b6.idx'
Traceback (most recent call last):
  File "c:\Users\Heitor\anaconda3\Lib\site-packages\cfgrib\messages.py", line 539, in from_indexpath_or_filestream
    self = cls.from_fieldset(filestream, index_keys, computed_keys)
  File "c:\Users\Heitor\anaconda3\Lib\site-packages\cfgrib\messages.py", line 379, in from_fieldset
    return cls.from_fieldset_and_iteritems(fieldset, iteritems, index_keys, computed_keys)
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Heitor\anaconda3\Lib\site-packages\cfgrib\messages.py", line 392, in from_fieldset_and_iteritems
    for field_id, raw_field in iteritems:
                               ^^^^^^^^^
  File "c:\Users\Heitor\anaconda3\Lib\site-packages\cfgrib\messages.py", line 292, in __iter__
    for message in self.itervalues():
                   ~~~~~~~~~~~~~~~^^
  File "c:\Users\Heitor

❌ Erro no download ENS Extended: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/retrieve/v1/processes/seasonal-subseasonal
process not found
dataset seasonal-subseasonal not found


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\Projetos\\previsao-do-tempo\\previsao\\ecmwf_rain_tp_run_20251029_00Z.grib2'